In [5]:
import pandas as pd
import datetime

from pandas.conftest import axis_1
from pyecharts.charts import *
from pyecharts import options as opts
df1 = pd.read_excel('./data/业务数据.xlsx')

In [7]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5257 entries, 0 to 5256
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   销售      5257 non-null   object 
 1   账单状态    5257 non-null   object 
 2   账单周期    5257 non-null   object 
 3   账单金额    5257 non-null   float64
 4   开票金额    5010 non-null   float64
 5   实收金额    4470 non-null   float64
 6   未收金额    5010 non-null   float64
 7   预计付款日   5256 non-null   object 
 8   应付日期    5257 non-null   object 
 9   商务催收日期  5257 non-null   object 
 10  账期      5257 non-null   int64  
 11  实际到账日   4387 non-null   object 
 12  开票日期    4996 non-null   object 
 13  客服      5257 non-null   object 
dtypes: float64(4), int64(1), object(9)
memory usage: 575.1+ KB


In [8]:
df1.describe()

,账单金额,开票金额,实收金额,未收金额,账期
count,5.257000e+03,5.010000e+03,4.470000e+03,5.010000e+03,5257.000000
mean,4.073241e+04,4.096896e+04,4.082419e+04,4.684636e+03,64.539661
std,8.176172e+04,8.007245e+04,7.970628e+04,2.888464e+04,15.622765
min,0.000000e+00,2.500000e+01,0.000000e+00,0.000000e+00,0.000000
25%,5.103000e+03,5.300000e+03,5.112250e+03,0.000000e+00,60.000000
50%,1.436500e+04,1.486560e+04,1.434000e+04,0.000000e+00,60.000000
75%,4.178000e+04,4.220250e+04,4.170750e+04,0.000000e+00,75.000000
max,1.508796e+06,1.356215e+06,1.301665e+06,1.277098e+06,90.000000


In [11]:
today_time = pd.to_datetime(df1['实际到账日'].fillna('0').max())

In [12]:
df1['实收金额'].fillna(0,inplace=True)
df1['未收金额'].fillna(0,inplace=True)

In [13]:
df1['账单周期'] = pd.to_datetime(df1['账单周期'])
df1['应付日期'] = pd.to_datetime(df1['应付日期'])
df1['实际到账日'] = pd.to_datetime(df1['实际到账日']).fillna(today_time)

In [15]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5257 entries, 0 to 5256
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   销售      5257 non-null   object        
 1   账单状态    5257 non-null   object        
 2   账单周期    5257 non-null   datetime64[ns]
 3   账单金额    5257 non-null   float64       
 4   开票金额    5010 non-null   float64       
 5   实收金额    5257 non-null   float64       
 6   未收金额    5257 non-null   float64       
 7   预计付款日   5256 non-null   object        
 8   应付日期    5257 non-null   datetime64[ns]
 9   商务催收日期  5257 non-null   object        
 10  账期      5257 non-null   int64         
 11  实际到账日   5257 non-null   datetime64[ns]
 12  开票日期    4996 non-null   object        
 13  客服      5257 non-null   object        
dtypes: datetime64[ns](3), float64(4), int64(1), object(6)
memory usage: 575.1+ KB


In [16]:
df1['是否到期'] = df1['应付日期'].apply(lambda x : 0 if x > today_time else 1)

In [17]:
df1['是否到期'].value_counts()

是否到期
1    4437
0     820
Name: count, dtype: int64

In [18]:
df1['账单周期'].value_counts()

账单周期
2018-11-01    428
2018-12-01    418
2019-03-01    416
2018-10-01    391
2019-04-01    382
2018-08-01    370
2019-01-01    358
2018-07-01    332
2018-09-01    320
2018-06-01    307
2018-05-01    279
2019-02-01    240
2018-04-01    222
2018-03-01    167
2018-01-01    124
2017-12-01    103
2018-02-01    102
2017-11-01     74
2017-09-01     58
2017-07-01     55
2017-10-01     55
2017-08-01     51
2017-06-01      3
2019-05-01      2
Name: count, dtype: int64

In [32]:
df1['是否到期90天'] = (today_time - df1['应付日期']).apply(lambda x : 1 if x.days >=90 else 0)

In [33]:
df1['是否到期90天'].value_counts()

是否到期90天
1    3374
0    1883
Name: count, dtype: int64

In [34]:
df1.drop('是否逾期90天', axis=1, inplace=True)

In [24]:
df1['未收金额2'] = (df1['账单金额'] - df1['实收金额'])

In [25]:
df1['历史逾期天数'] = df1.apply(lambda x : (x['实际到账日'] - x['应付日期']).days if x['未收金额2'] == 0 else (today_time - x['应付日期']).days,axis=1)

In [26]:
df1['历史逾期天数']

0       -75
1       -75
2       -44
3       -75
4       -75
       ... 
5252    624
5253    594
5254      0
5255     -9
5256    624
Name: 历史逾期天数, Length: 5257, dtype: int64

In [27]:
df1['当前逾期天数'] = df1.apply(lambda x : x['历史逾期天数'] if x['未收金额2'] > 0 else 0,axis=1)

In [29]:
df1['账单季度'] = df1['账单周期'].apply(lambda x : x.to_period('Q'))
df1['账单季度'].value_counts()

账单季度
2018Q4    1237
2018Q3    1022
2019Q1    1014
2018Q2     808
2018Q1     393
2019Q2     384
2017Q4     232
2017Q3     164
2017Q2       3
Freq: Q-DEC, Name: count, dtype: int64

In [35]:
df2 = df1[(df1['账单季度'] <= '2018Q4') & (df1['账单季度'] >= '2017Q3')]
df2.shape

(3856, 20)

In [36]:
fn1 = df2.groupby('账单季度')[['账单金额']].sum()
fn1

,账单金额
账单季度,
2017Q3,8247952.62
2017Q4,11643604.99
2018Q1,17149674.79
2018Q2,31097661.29
2018Q3,38292071.12
2018Q4,51963089.64


In [39]:
df3 = df2[df1['是否到期90天'] == 1]

C:\Users\46901\AppData\Local\Temp\ipykernel_12624\3548058834.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df3 = df2[df1['是否到期90天'] == 1]


In [43]:
fn2 = df3.groupby('账单季度')[['账单金额']].sum()
fn2.columns = ['到期金额']

In [44]:
fn2

,到期金额
账单季度,
2017Q3,8247952.62
2017Q4,11643604.99
2018Q1,17149674.79
2018Q2,31097661.29
2018Q3,38292071.12
2018Q4,28265677.59


In [45]:
fn3 = df3.groupby('账单季度')[['未收金额2']].sum()
fn3.columns = ['当前逾期90+金额']
fn3

,当前逾期90+金额
账单季度,
2017Q3,63883.0
2017Q4,57380.0
2018Q1,64283.0
2018Q2,106930.0
2018Q3,412920.1
2018Q4,304183.0


In [48]:
final1 = pd.concat([fn1, fn2, fn3], axis=1)

In [52]:
final1['90+净坏账率'] = round(final1['当前逾期90+金额'] / final1['到期金额'],3)

In [53]:
final1

,账单金额,到期金额,当前逾期90+金额,90+净坏账率
账单季度,,,,
2017Q3,8247952.62,8247952.62,63883.0,0.008
2017Q4,11643604.99,11643604.99,57380.0,0.005
2018Q1,17149674.79,17149674.79,64283.0,0.004
2018Q2,31097661.29,31097661.29,106930.0,0.003
2018Q3,38292071.12,38292071.12,412920.1,0.011
2018Q4,51963089.64,28265677.59,304183.0,0.011


In [54]:
bar = (
    Bar()
    .add_xaxis(list(final1.index.values.astype(str)))
    .add_yaxis(
        "账单金额",
        list(final1.账单金额),
        yaxis_index=0,
        color="#5793f3",
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="90+净坏账率"),
    )
    .extend_axis(
        yaxis=opts.AxisOpts(
            name="90+净坏账率",
            type_="value",
            min_=0,
            max_=0.014,
            position="right",
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(color="#d14a61")
            ),
            axislabel_opts=opts.LabelOpts(formatter="{value}"),
        )
    )
)
line = (
    Line()
    .add_xaxis(list(final1.index.values.astype(str)))
    .add_yaxis(
        "90+净坏账率",
        list(final1['90+净坏账率']),
        yaxis_index=1,
        color="#675bba",
        label_opts=opts.LabelOpts(is_show=False),
    )
)
bar.overlap(line).render()

'D:\\python-learn\\风控项目\\render.html'

In [57]:
df4 = df1[(df1['账期'] == 60) & df1['是否到期'] == 1]
fn1 = df4.groupby('账单季度')[['账单金额']].sum()
fn1.columns = ['60天账期的账单金额']

In [59]:
df5 = df1[(df1['账期'] == 60) & df1['是否到期'] == 1 & (df1['历史逾期天数'] > 0)]
fn2 = df5.groupby('账单季度')[['未收金额2']].sum()
fn2.columns = ['60天账期的入催金额']
fn2

,60天账期的入催金额
账单季度,
2017Q2,34550.00
2017Q3,36983.00
2017Q4,52750.00
2018Q1,62460.00
2018Q2,13590.00
2018Q3,380265.10
2018Q4,584789.50
2019Q1,13695563.96
2019Q2,14721753.49


In [60]:
df6 = df1[(df1['账期'] == 90) & df1['是否到期'] == 1]
fn3 = df6.groupby('账单季度')[['账单金额']].sum()
fn3.columns = ['90天账期的账单金额']
df7 = df1[(df1['账期'] == 90) & df1['是否到期'] == 1 & (df1['历史逾期天数'] > 0)]
fn4 = df7.groupby('账单季度')[['未收金额2']].sum()
fn4.columns = ['90天账期的入催金额']
fn4

,90天账期的入催金额
账单季度,
2017Q2,0.00
2017Q3,1900.00
2017Q4,0.00
2018Q1,800.00
2018Q2,0.00
2018Q3,8235.00
2018Q4,325141.00
2019Q1,13382174.96
2019Q2,14721753.49


In [62]:
final2 = pd.concat([fn1, fn2, fn3, fn4], axis=1)

In [63]:
final2 = final2[(final2.index <= '2018Q4') & (final2.index >= '2017Q3')]

In [65]:
final2['60天账期的入催率'] = final2['60天账期的入催金额']/final2['60天账期的账单金额']
final2['90天账期的入催率'] = final2['90天账期的入催金额']/final2['90天账期的账单金额']

In [66]:
final2

,60天账期的账单金额,60天账期的入催金额,90天账期的账单金额,90天账期的入催金额,60天账期的入催率,90天账期的入催率
账单季度,,,,,,
2017Q3,4854770.94,36983.0,2769264.0,1900.0,0.007618,0.000686
2017Q4,6737327.99,52750.0,3921491.0,0.0,0.007830,0.000000
2018Q1,12106356.79,62460.0,4244304.0,800.0,0.005159,0.000188
2018Q2,19234086.87,13590.0,8427775.0,0.0,0.000707,0.000000
2018Q3,22830710.42,380265.1,9835629.0,8235.0,0.016656,0.000837
2018Q4,26337959.52,584789.5,17706430.0,325141.0,0.022203,0.018363


In [68]:
line = (
    Line()
    .add_xaxis(list(final2.index.values.astype(str)))
    .add_yaxis(
        "60天账期入催率",
        list(final2['60天账期的入催率']),
        yaxis_index=0,
        color="#675bba",
        label_opts=opts.LabelOpts(is_show=False),
    )
    .set_global_opts(
    title_opts=opts.TitleOpts(title="不同账期入催率"),
    )
    .add_xaxis(list(final2.index.values.astype(str)))
    .add_yaxis(
        "90天账期入催率",
        list(final2['90天账期的入催率']),
        yaxis_index=0,
        color="#d14a61",
        label_opts=opts.LabelOpts(is_show=False),
    )
)
line.render()

'D:\\python-learn\\风控项目\\render.html'

In [69]:
df2

,销售,账单状态,账单周期,账单金额,开票金额,实收金额,未收金额,预计付款日,应付日期,商务催收日期,账期,实际到账日,开票日期,客服,是否到期,未收金额2,历史逾期天数,当前逾期天数,账单季度,是否到期90天
1398,s1499,已确认,2018-12-01,158695.0,158695.0,0.0,158695.0,2019-05-16,2019-02-28,2019-03-15,60,2019-05-17,03-21,a1599,1,158695.0,78,78,2018Q4,0
1399,s1500,已确认,2018-12-01,132310.0,132310.0,0.0,132310.0,2019-04-05,2019-03-31,2019-04-15,90,2019-05-17,01-17,a1600,1,132310.0,47,47,2018Q4,0
1400,s1501,已确认,2018-12-01,103570.0,103570.0,0.0,103570.0,2019-05-17,2019-02-28,2019-03-15,60,2019-05-17,01-25,a1601,1,103570.0,78,78,2018Q4,0
1401,s1502,已确认,2018-12-01,55333.0,55333.0,0.0,55333.0,2019-05-10,2019-03-31,2019-04-15,90,2019-05-17,03-01,a1602,1,55333.0,47,47,2018Q4,0
1402,s1503,部分核销,2018-12-01,42710.0,42710.0,0.0,42710.0,2019-05-16,2019-02-28,2019-03-15,60,2019-04-09,01-30,a1603,1,42710.0,78,78,2018Q4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5249,s5350,已核销,2017-07-01,12450.0,12450.0,12450.0,0.0,2017-09-30,2017-09-30,2017-10-15,60,2017-10-11,11-07,a5450,1,0.0,11,0,2017Q3,1
5250,s5351,已确认,2017-07-01,1900.0,NaN,0.0,0.0,2017-10-31,2017-10-31,2017-11-15,90,2019-05-17,NaN,a5451,1,1900.0,563,563,2017Q3,1
5251,s5352,已确认,2017-07-01,11500.0,NaN,0.0,0.0,2017-09-30,2017-09-30,2017-10-15,60,2019-05-17,NaN,a5452,1,11500.0,594,594,2017Q3,1
5252,s5353,已确认,2017-07-01,22800.0,NaN,0.0,0.0,2017-07-31,2017-08-31,2017-09-15,30,2019-05-17,NaN,a5453,1,22800.0,624,624,2017Q3,1


In [80]:
df8 = df2[(df2['未收金额2'] == 0) & (df2['是否到期'] == 1)]

In [81]:
df8['历史逾期天数'].describe()

count    3793.000000
mean        4.350910
std        43.300838
min      -123.000000
25%        -8.000000
50%        -1.000000
75%         8.000000
max       471.000000
Name: 历史逾期天数, dtype: float64

In [83]:
df8['历史逾期天数2'] = pd.cut(df8['历史逾期天数'],bins=[-999,0,7,15,30,60,90,999],labels=['0','1-7','8-15','16-30','31-60','61-90','90+'])

C:\Users\46901\AppData\Local\Temp\ipykernel_12624\18288406.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df8['历史逾期天数2'] = pd.cut(df8['历史逾期天数'],bins=[-999,0,7,15,30,60,90,999],labels=['0','1-7','8-15','16-30','31-60','61-90','90+'])


In [84]:
df8['历史逾期天数2'].value_counts()

历史逾期天数2
0        2400
1-7       440
8-15      368
16-30     281
31-60     156
90+        88
61-90      60
Name: count, dtype: int64

In [86]:
final3 = df8.groupby('历史逾期天数2')[['账期']].count()
final3.columns = ['回收账单数']
final3

,回收账单数
历史逾期天数2,
0,2400
1-7,440
8-15,368
16-30,281
31-60,156
61-90,60
90+,88


In [87]:
ydata = final3['回收账单数'].values.tolist()
bar = (
    Bar()
    .add_xaxis(list(final3.index.values.tolist()))
    .add_yaxis("收回账单数",ydata,yaxis_index=0,color="#675bba")
    .set_global_opts(
    title_opts=opts.TitleOpts(title="不同逾期天数的已收回账单数"),
    )
)
bar.render()


'D:\\python-learn\\风控项目\\render.html'